# Подключение к диску

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


# Загрузка данных

In [ ]:
import numpy as np   # для работы с массивами
import pandas as pd  # для работы с файлами csv

print('Обучающая выборка')
train_data = pd.read_csv("/content/drive/MyDrive/titanic/train.csv")
train_data.head()


Обучающая выборка


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
print('Тестовая выборка')
test_data = pd.read_csv("/content/drive/MyDrive/titanic/test.csv")
test_data.head()

Тестовая выборка


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Подготовка атрибутов к анализу

In [ ]:
train_data["familysize"]=(train_data['SibSp']+train_data['Parch']).astype(str)
# получаем индикаторные столбцы
data = pd.get_dummies(train_data[["Sex","Embarked","familysize"]])

data.head()

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,familysize_0,familysize_1,familysize_10,familysize_2,familysize_3,familysize_4,familysize_5,familysize_6,familysize_7
0,0,1,0,0,1,0,1,0,0,0,0,0,0,0
1,1,0,1,0,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,1,1,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,0,0,0,0,0
4,0,1,0,0,1,1,0,0,0,0,0,0,0,0


In [ ]:
data["class1"]= np.where((train_data['Pclass']==1), 1, 0)
data["class2"]= np.where((train_data['Pclass']==2), 1, 0)
data["class3"]= np.where((train_data['Pclass']==3), 1, 0)


In [ ]:
# замена пропущенных значений возраста

mean_age = train_data["Age"].mean() # средний возраст
print('Средний возраст')
print(mean_age)

train_data["Age"] = train_data["Age"].fillna(mean_age)

test_data["Age"] = test_data["Age"].fillna(mean_age)


Средний возраст
29.69911764705882


In [ ]:
data["age1"]= np.where((train_data['Age'] <=16), 1, 0)
data["age2"]=np.where((train_data.Age >16) & (train_data.Age <mean_age),1,0)
data["age3"]=np.where((train_data.Age >mean_age) & (train_data.Age <mean_age*2),1,0)
data["age4"]=np.where( (train_data.Age >=mean_age*2),1,0)

In [ ]:
# замена пропущенных значений цены билета

mean_fare = train_data["Fare"].mean() # средняя цена
print('Средняя цена билета')
print(mean_fare)

train_data["Fare"] = train_data["Fare"].fillna(mean_fare)
test_data["Fare"] = test_data["Fare"].fillna(mean_fare)

Средняя цена билета
32.204207968574636


In [ ]:
data["fare1"]= np.where((train_data['Fare'] <=mean_fare/2), 1, 0)
data["fare2"]=np.where((train_data.Fare >mean_fare/2) & (train_data.Fare <mean_fare),1,0)
data["fare3"]=np.where((train_data.Fare >mean_fare) & (train_data.Fare <mean_fare*2),1,0)
data["fare4"]=np.where( (train_data.Fare >=mean_fare*2),1,0)

In [ ]:
data["master"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Master') , 1,0)
data["mr"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Mr') , 1,0)
data["mrs"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Mrs') , 1,0)
data["miss"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Miss') , 1,0)

# Анализ корреляции атрибутов со столбцом Survived

In [ ]:
data["SURVIVED"]=train_data["Survived"]

# печатаем  корреляции со столбцом Survived
print(data.corr()['SURVIVED'])

Sex_female       0.543351
Sex_male        -0.543351
Embarked_C       0.168240
Embarked_Q       0.003650
Embarked_S      -0.155660
familysize_0    -0.203367
familysize_1     0.163157
familysize_10   -0.070234
familysize_2     0.143869
familysize_3     0.128347
familysize_4    -0.049466
familysize_5    -0.080968
familysize_6    -0.012134
familysize_7    -0.064988
class1           0.285904
class2           0.093349
class3          -0.322308
age1             0.121485
age2            -0.039670
age3             0.050201
age4            -0.040857
fare1           -0.276811
fare2            0.082939
fare3            0.070220
fare4            0.243110
master           0.085221
mr              -0.549199
mrs              0.339040
miss             0.327093
SURVIVED         1.000000
Name: SURVIVED, dtype: float64


In [ ]:
# столбцы по размеру семьи

data["lonely"]= np.where((train_data['SibSp']+train_data['Parch'])==0, 1, 0)
data["small_family"]= np.where(((train_data['SibSp']+train_data['Parch'])>0) & ((train_data['SibSp']+train_data['Parch'])<=3) , 1, 0)
data["big_family"]= np.where((train_data['SibSp']+train_data['Parch'])>3, 1, 0)

In [ ]:
# печатаем  корреляции со столбцом Survived
print(data.corr()['SURVIVED'])

Sex_female       0.543351
Sex_male        -0.543351
Embarked_C       0.168240
Embarked_Q       0.003650
Embarked_S      -0.155660
familysize_0    -0.203367
familysize_1     0.163157
familysize_10   -0.070234
familysize_2     0.143869
familysize_3     0.128347
familysize_4    -0.049466
familysize_5    -0.080968
familysize_6    -0.012134
familysize_7    -0.064988
class1           0.285904
class2           0.093349
class3          -0.322308
age1             0.121485
age2            -0.039670
age3             0.050201
age4            -0.040857
fare1           -0.276811
fare2            0.082939
fare3            0.070220
fare4            0.243110
master           0.085221
mr              -0.549199
mrs              0.339040
miss             0.327093
SURVIVED         1.000000
lonely          -0.203367
small_family     0.279855
big_family      -0.125147
Name: SURVIVED, dtype: float64


# Создание и тестирование модели

In [ ]:

# атрибуты для обучения
data = pd.get_dummies(train_data[["Sex"]])
data["C"] = np.where((train_data["Embarked"]=='C'),1,0)
data["S"] = np.where((train_data["Embarked"]=='S'),1,0)
data["class1"]= np.where((train_data['Pclass']==1), 1, 0)
data["class2"]= np.where((train_data['Pclass']==2), 1, 0)
data["class3"]= np.where((train_data['Pclass']==3), 1, 0)
data["age1"]= np.where((train_data['Age'] <=16), 1, 0)
data["fare1"]= np.where((train_data['Fare'] <=mean_fare/2), 1, 0)
data["fare4"]=np.where( (train_data.Fare >=mean_fare*2),1,0)
data["mr"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Mr') , 1,0)
data["mrs"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Mrs') , 1,0)
data["miss"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Miss') , 1,0)
data["lonely"]= np.where((train_data['SibSp']+train_data['Parch'])==0, 1, 0)
data["small_family"]= np.where(((train_data['SibSp']+train_data['Parch'])>0) & ((train_data['SibSp']+train_data['Parch'])<=3) , 1, 0)
data["big_family"]= np.where((train_data['SibSp']+train_data['Parch'])>3, 1, 0)

# атрибуты для тестирования
tdata = pd.get_dummies(test_data[["Sex"]])
tdata["C"] = np.where((test_data["Embarked"]=='C'),1,0)
tdata["S"] = np.where((test_data["Embarked"]=='S'),1,0)
tdata["class1"]= np.where((test_data['Pclass']==1), 1, 0)
tdata["class2"]= np.where((test_data['Pclass']==2), 1, 0)
tdata["class3"]= np.where((test_data['Pclass']==3), 1, 0)
tdata["age1"]= np.where((test_data['Age'] <=16), 1, 0)
tdata["fare1"]= np.where((test_data['Fare'] <=mean_fare/2), 1, 0)
tdata["fare4"]=np.where( (test_data.Fare >=mean_fare*2),1,0)
tdata["mr"]= np.where( ((test_data.Name).str.partition('.')[0]).str.endswith('Mr') , 1,0)
tdata["mrs"]= np.where( ((test_data.Name).str.partition('.')[0]).str.endswith('Mrs') , 1,0)
tdata["miss"]= np.where( ((test_data.Name).str.partition('.')[0]).str.endswith('Miss') , 1,0)
tdata["lonely"]= np.where((test_data['SibSp']+test_data['Parch'])==0, 1, 0)
tdata["small_family"]= np.where(((test_data['SibSp']+test_data['Parch'])>0) & ((test_data['SibSp']+test_data['Parch'])<=3) , 1, 0)
tdata["big_family"]= np.where((test_data['SibSp']+test_data['Parch'])>3, 1, 0)

In [ ]:
data["chance"] = (data["Sex_female"]*0.54 - data["Sex_male"]*0.54
                  +data["C"]*0.17-data["S"]*0.16+data["class1"]*0.29
                  +data["class2"]*0.09-data["class3"]*0.32+ data["age1"]*0.12
                  - data['fare1']*0.28+data['fare4']*0.24-data['lonely']*0.2
                  + data['small_family']*0.28-data['big_family']*0.13
                  -data["mr"]*0.55+data["mrs"]*0.34+data["miss"]*0.33)

In [ ]:
tdata["chance"] = (tdata["Sex_female"]*0.54 - tdata["Sex_male"]*0.54
                  +tdata["C"]*0.17-tdata["S"]*0.16+tdata["class1"]*0.29
                  +tdata["class2"]*0.09-tdata["class3"]*0.32+ tdata["age1"]*0.12
                  - tdata['fare1']*0.28+tdata['fare4']*0.24-tdata['lonely']*0.2
                  + tdata['small_family']*0.28-tdata['big_family']*0.13
                  -tdata["mr"]*0.55+tdata["mrs"]*0.34+tdata["miss"]*0.33)

In [ ]:
# гауссовский наивный байесовский классификатор
from sklearn.naive_bayes import GaussianNB

y = train_data['Survived'] # метки

In [ ]:
model = GaussianNB() # создаём модель
model.fit(data[['chance']], y) # обучаем
# результат на обучающей выборке
display(model.score(data[['chance']], y))


0.7901234567901234

# Анализ корреляции атрибутов с учетом пола пассажира

In [ ]:
data = pd.get_dummies(train_data[["Sex","Embarked"]])
data["class1"]= np.where((train_data['Pclass']==1), 1, 0)
data["class2"]= np.where((train_data['Pclass']==2), 1, 0)
data["class3"]= np.where((train_data['Pclass']==3), 1, 0)
data["age1"]= np.where((train_data['Age'] <=16), 1, 0)
data["age2"]=np.where((train_data.Age >16) & (train_data.Age <mean_age),1,0)
data["age3"]=np.where((train_data.Age >mean_age) & (train_data.Age <mean_age*2),1,0)
data["age4"]=np.where( (train_data.Age >=mean_age*2),1,0)
data["fare1"]= np.where((train_data['Fare'] <=mean_fare/2), 1, 0)
data["fare2"]=np.where((train_data.Fare >mean_fare/2) & (train_data.Fare <mean_fare),1,0)
data["fare3"]=np.where((train_data.Fare >mean_fare) & (train_data.Fare <mean_fare*2),1,0)
data["fare4"]=np.where( (train_data.Fare >=mean_fare*2),1,0)
data["master"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Master') , 1,0)
data["mr"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Mr') , 1,0)
data["mrs"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Mrs') , 1,0)
data["miss"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Miss') , 1,0)
data["lonely"]= np.where((train_data['SibSp']+train_data['Parch'])==0, 1, 0)
data["small_family"]= np.where(((train_data['SibSp']+train_data['Parch'])>0) & ((train_data['SibSp']+train_data['Parch'])<=3) , 1, 0)
data["big_family"]= np.where((train_data['SibSp']+train_data['Parch'])>3, 1, 0)

# выжившие женщины
data["female_survived"]=train_data.loc[train_data.Sex == 'female']["Survived"]

# выжившие мужчины
data["male_survived"]=train_data.loc[train_data.Sex == 'male']["Survived"]

print("Влияние на выживаемость женщин")
print(data.corr()["female_survived"])

print('')

print("Влияние на выживаемость мужчин")
print(data.corr()["male_survived"])

Влияние на выживаемость женщин
Sex_female              NaN
Sex_male                NaN
Embarked_C         0.169413
Embarked_Q         0.006549
Embarked_S        -0.161915
class1             0.337723
class2             0.231215
class3            -0.509155
age1              -0.067392
age2              -0.011307
age3               0.097282
age4               0.066975
fare1             -0.158651
fare2             -0.084936
fare3              0.054252
fare4              0.235028
master                  NaN
mr                      NaN
mrs                0.092869
miss              -0.118723
lonely             0.081726
small_family       0.145363
big_family        -0.367598
female_survived    1.000000
male_survived           NaN
Name: female_survived, dtype: float64

Влияние на выживаемость мужчин
Sex_female              NaN
Sex_male                NaN
Embarked_C         0.131966
Embarked_Q        -0.081775
Embarked_S        -0.065808
class1             0.238041
class2            -0.038618
cla

# Создание улучшенной модели

In [ ]:
# атрибуты для обучения
data = pd.get_dummies(train_data[["Sex"]])
data["C"] = np.where((train_data["Embarked"]=='C'),1,0)
data["S"] = np.where((train_data["Embarked"]=='S'),1,0)
data["class1"]= np.where((train_data['Pclass']==1), 1, 0)
data["class2"]= np.where((train_data['Pclass']==2), 1, 0)
data["class3"]= np.where((train_data['Pclass']==3), 1, 0)
data["age1"]= np.where((train_data['Age'] <=16), 1, 0)
data["age3"]=np.where((train_data.Age >mean_age) & (train_data.Age <mean_age*2),1,0)
data["fare1"]= np.where((train_data['Fare'] <=mean_fare/2), 1, 0)
data["fare2"]=np.where((train_data.Fare >mean_fare/2) & (train_data.Fare <mean_fare),1,0)
data["fare3"]=np.where((train_data.Fare >mean_fare) & (train_data.Fare <mean_fare*2),1,0)
data["fare4"]=np.where( (train_data.Fare >=mean_fare*2),1,0)
data["master"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Master') , 1,0)
data["mr"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Mr') , 1,0)
data["mrs"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Mrs') , 1,0)
data["miss"]= np.where( ((train_data.Name).str.partition('.')[0]).str.endswith('Miss') , 1,0)
data["lonely"]= np.where((train_data['SibSp']+train_data['Parch'])==0, 1, 0)
data["small_family"]= np.where(((train_data['SibSp']+train_data['Parch'])>0) & ((train_data['SibSp']+train_data['Parch'])<=3) , 1, 0)
data["big_family"]= np.where((train_data['SibSp']+train_data['Parch'])>3, 1, 0)

# атрибуты для тестирования
tdata = pd.get_dummies(test_data[["Sex"]])
tdata["C"] = np.where((test_data["Embarked"]=='C'),1,0)
tdata["S"] = np.where((test_data["Embarked"]=='S'),1,0)
tdata["class1"]= np.where((test_data['Pclass']==1), 1, 0)
tdata["class2"]= np.where((test_data['Pclass']==2), 1, 0)
tdata["class3"]= np.where((test_data['Pclass']==3), 1, 0)
tdata["age1"]= np.where((test_data['Age'] <=16), 1, 0)
tdata["age3"]=np.where((test_data.Age >mean_age) & (test_data.Age <mean_age*2),1,0)
tdata["fare1"]= np.where((test_data['Fare'] <=mean_fare/2), 1, 0)
tdata["fare2"]=np.where((test_data.Fare >mean_fare/2) & (test_data.Fare <mean_fare),1,0)
tdata["fare3"]=np.where((test_data.Fare >mean_fare) & (test_data.Fare <mean_fare*2),1,0)
tdata["fare4"]=np.where( (test_data.Fare >=mean_fare*2),1,0)
tdata["master"]= np.where( ((test_data.Name).str.partition('.')[0]).str.endswith('Master') , 1,0)
tdata["mr"]= np.where( ((test_data.Name).str.partition('.')[0]).str.endswith('Mr') , 1,0)
tdata["mrs"]= np.where( ((test_data.Name).str.partition('.')[0]).str.endswith('Mrs') , 1,0)
tdata["miss"]= np.where( ((test_data.Name).str.partition('.')[0]).str.endswith('Miss') , 1,0)
tdata["lonely"]= np.where((test_data['SibSp']+test_data['Parch'])==0, 1, 0)
tdata["small_family"]= np.where(((test_data['SibSp']+test_data['Parch'])>0) & ((test_data['SibSp']+test_data['Parch'])<=3) , 1, 0)
tdata["big_family"]= np.where((test_data['SibSp']+test_data['Parch'])>3, 1, 0)

In [ ]:
data["chance"]=(data["Sex_female"]*(0.54+0.1*data['age3']-0.12*data['miss']
                                    +0.09*data['mrs']+data['C']*0.17-data['S']*0.16
                                    +data['small_family']*0.15-data["big_family"]*0.37
                                    +data['class1']*0.34+data['class2']*0.23-data['class3']*0.51
                                    -data['fare1']*0.16+data['fare4']*0.24)
                                    +data["Sex_male"]*(-0.54+0.27*data["master"]-0.24*data["mr"]
                                                       +0.13*data['C']-0.13*data['lonely']
                                                       +0.19*data['small_family']-0.09*data['big_family']
                                                       +0.24*data['class1']-0.17*data['class3']
                                                       +0.19*data['age1']-0.22*data['fare1']+0.13*data['fare2']
                                                       +0.11*data['fare3']))

tdata["chance"]=(tdata["Sex_female"]*(0.54+0.1*tdata['age3']-0.12*tdata['miss']
                                    +0.09*tdata['mrs']+tdata['C']*0.17-tdata['S']*0.16
                                    +tdata['small_family']*0.15-tdata["big_family"]*0.37
                                    +tdata['class1']*0.34+tdata['class2']*0.23-tdata['class3']*0.51
                                    -tdata['fare1']*0.16+tdata['fare4']*0.24)
                                    +tdata["Sex_male"]*(-0.54+0.27*tdata["master"]-0.24*tdata["mr"]
                                                       +0.13*tdata['C']-0.13*tdata['lonely']
                                                       +0.19*tdata['small_family']-0.09*tdata['big_family']
                                                       +0.24*tdata['class1']-0.17*tdata['class3']
                                                       +0.19*tdata['age1']-0.22*tdata['fare1']+0.13*tdata['fare2']
                                                       +0.11*tdata['fare3']))

In [ ]:
# гауссовский наивный байесовский классификатор
from sklearn.naive_bayes import GaussianNB

y = train_data['Survived'] # метки

In [ ]:
model = GaussianNB() # создаём модель
model.fit(data[['chance']], y) # обучаем
# результат на обучающей выборке
display(model.score(data[['chance']], y))

0.8148148148148148

In [ ]:
predictions = model.predict(tdata[['chance']])

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('try.csv', index=False)